In [1]:
import tensorflow as tf 
import numpy as np 

much_data = np.load('training_np--30--30.npy')
train_data = much_data[:-15]
validation_data = much_data[-15:]

In [2]:
much_data

array([[ array([[128, 126, 125, 123, 123, 126, 129, 129, 128, 127, 126, 126, 126,
        128, 129, 128, 126, 123, 124, 126, 128, 127, 125, 125, 127, 129,
        130, 130, 129, 129],
       [127, 126, 125, 124, 124, 126, 129, 129, 129, 128, 127, 127, 127,
        129, 130, 129, 127, 125, 125, 127, 128, 127, 125, 125, 127, 129,
        130, 130, 129, 129],
       [126, 126, 125, 125, 126, 127, 129, 130, 131, 131, 130, 130, 130,
        131, 132, 131, 130, 128, 128, 128, 128, 128, 126, 126, 127, 128,
        129, 129, 129, 129],
       [125, 125, 126, 128, 129, 129, 130, 131, 133, 134, 134, 133, 133,
        133, 134, 134, 134, 133, 133, 131, 130, 129, 128, 128, 128, 127,
        128, 128, 128, 128],
       [126, 127, 128, 130, 131, 131, 132, 133, 134, 137, 137, 138, 138,
        138, 139, 139, 141, 140, 138, 135, 132, 131, 130, 130, 128, 127,
        127, 128, 128, 128],
       [128, 129, 130, 132, 132, 132, 133, 133, 134, 137, 138, 140, 141,
        142, 143, 144, 145, 144, 142, 138, 

In [3]:


sess = tf.InteractiveSession()
#build a softmax regression model
# shape arg is optional, lets tf automatically catch bugs from inconsistent tensorshapes
x = tf.placeholder(tf.float32, shape=[None, 30, 30], name = "x") # 28x28 pixel image = 784, None indicates frst dimension
y_ = tf.placeholder(tf.float32, shape = [None, 2], name = "y") #, shape = [2 ]) # , shape = [None, 2]) # one hot 10-dimensional vector indictaing which digit class (0-9)


#
###
###### MULTILAYER COMPUTATIONAL GRAPH

# TODO. Beautify tensorbaord by using tf.scope and wrapping names around the weight and bias variables 

def weight_variable(shape, w_name):
    initial = tf.truncated_normal(shape, stddev=0.1)

    return tf.Variable(initial, name = 'NN'+str(w_name)+'/W')

# add bias 0.1 to avoid 'dead neurons' from using ReLU neurons/ activation functions
def bias_variable(shape, b_name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name = 'NN'+str(b_name)+'/B')

# convolution and pooling. handling the boundaries and stride sizes. This is the vanilla vesion (stride of 1, zero padding, output is same size as input)
# pooling is plain old max pooling over 2x2 blocks
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')



In [4]:
# first convolutional layer. consists of our first convolution, followed by max pooling
# convolution will compute 32 features for each 5x5 patch
# weight tensor wil thus have shape of [5, 5, 1, 32] i.e. [ patch size, patch size, input channels, output channels]
with tf.variable_scope('INZ'):
    W_conv1 = weight_variable([5, 5, 1, 32], 1)
    b_conv1 = bias_variable([32], 1) # bias vector

# reshape x to a 4d tensor, with the 2nd and 3rd dimensions corresponding to image width and height, last dimension is # of color channels
x_image = tf.reshape(x, [-1, 30, 30, 1])

# convolve x_image with the weight tesnor, add bias, apply ReLU function, then max pool
# max pool_2x2 resizes image size to 14x14
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer 
# will have 64 features for each 5x5 patch
W_conv2 = weight_variable([5, 5, 32, 64], 2)
b_conv2 = bias_variable([64], 2)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# densely connected layer. Image size has been reduced to 7x7, we add a full_connected layer with 1024 neurons to allow processing on the entire image
# reshape the tensor from the pooling layer into a batch of vectors, multiple by a weight matrix, add bias, apply ReLU
W_fc1 = weight_variable([8 * 8 * 64, 1024], 'fc1')
b_fc1 = bias_variable([1024], 'fc1')

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# add dropout to reduce overfitting - before readout layer. Dropout only really useful in large CNNs
# turn dropout on during training, turn it off during testing
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
W_fc2 = weight_variable([1024, 2], 'fc2')
b_fc2 = bias_variable([2], 'fc2')

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2



In [5]:
# train and evalutate the model
# 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y_, logits=y_conv))
optimizer = tf.train.AdamOptimizer(1e-6).minimize(cross_entropy) # step size is very sensitive
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

epochs = 20

epoch_loss = 0 

logs_path = "logs/"
# train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train', sess.graph)
with tf.Session() as sess:
    
    # writes to TensorBoard 
    writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
    
    # run session, initialise variables
    sess.run(tf.initialize_all_variables())
    
    for epoch in range(epochs):
        for i in range(0, len(train_data), 20):
    
            d = train_data[i: i+20]
            
            X, Y = zip(*d)
     
            X = np.stack(list(X), axis=0)
            Y = np.vstack(list(Y))

            X = X.astype(float)
            Y = Y.astype(float)
            
            a = np.transpose(Y)
            
            c = list()

            for i in Y: 
                if i == 1:
                    c.append(np.array([0.0,1.0]))
                if i == 0:
                    c.append(np.array([1.0,0.0]))

            Y = np.vstack(c)
            
            print("print: ", sess.run(cross_entropy, feed_dict={x: X , y_ : Y, keep_prob: 1.0}))
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x:X, y_:Y, keep_prob: 0.9})
#             print c, _
            
            
    print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)

#         if i % 100 == 0:
#             train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
#             print('step %d, training accuracy %g' % (i, train_accuracy))
#             optimizer.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0}))
    
    # todo
    # store accuracy in a vector
    # store entropy in vetor
    # graph the vectors
    

('print: ', 381.51767)
('print: ', 500.40762)
('print: ', 328.15063)
('print: ', 282.42096)
('print: ', 317.52826)
('print: ', 351.41345)
('print: ', 230.33243)
('print: ', 377.49896)
('print: ', 371.03638)
('print: ', 328.40689)
('print: ', 430.35587)
('print: ', 281.72064)
('print: ', 242.02031)
('print: ', 271.64398)
('print: ', 299.98822)
('print: ', 196.19426)
('print: ', 321.00702)
('print: ', 314.78934)
('print: ', 277.55362)
('print: ', 362.25351)
('print: ', 236.22319)
('print: ', 202.18408)
('print: ', 226.08716)
('print: ', 248.72725)
('print: ', 162.05913)
('print: ', 264.20306)
('print: ', 258.24908)
('print: ', 227.0275)
('print: ', 295.4704)
('print: ', 191.92456)
('print: ', 163.59473)
('print: ', 182.2197)
('print: ', 199.65152)
('print: ', 129.65337)
('print: ', 210.57211)
('print: ', 205.01341)
('print: ', 179.43506)
('print: ', 232.4617)
('print: ', 150.49159)
('print: ', 127.90686)
('print: ', 142.02414)
('print: ', 155.26637)
('print: ', 100.59444)
('print: ', 163